# Basic Corountine Example

Jupyter notebook里面已经有一条 `event_loop`, `asyncio.run()`会强制再建一条. 可能会报错

In [3]:
import asyncio
import time

In [10]:
async def fetch_data(url: str, delay: float = 1.0) -> str:
    """
    Simulates fetching data from a URL with network delay.
    
    This is a COROUTINE FUNCTION because it's defined with 'async def'.
    When called, it doesn't execute immediately - it returns a coroutine object.
    """
    print(f"Fetching data from {url}...")
    await asyncio.sleep(delay)  # Simulate network delay

    return f"Data from {url}"



async def process_data(data: str) -> str:
    """
    Simulates processing data with computation delay.
    
    Another coroutine function that demonstrates chaining async operations.
    """
    print(f"Processing: {data}")
    
    # Another await - this coroutine will pause here
    # The event loop can run other tasks during this delay ⭐️
    await asyncio.sleep(1)  # Simulate processing time
    
    # Return the processed result
    return f"Processed: {data}"


async def basic_coroutine_example():
    """
    Example of basic coroutine usage with await.
    
    This demonstrates the fundamental pattern of async programming:
    1. Call async function with await
    2. Use the returned result
    3. Chain multiple async operations
    """
    
    # AWAIT PATTERN 1: Sequential execution
    # This line does several things:
    # 1. Calls fetch_data() which returns a coroutine object
    # 2. 'await' tells the event loop to run that coroutine ⭐️
    # 3. This function pauses until fetch_data() completes
    # 4. The result is assigned to 'data'
    print("Step 1: Starting data fetch...")
    data = await fetch_data("https://api.example.com")
    print(f"Step 1 complete: {data}")
    
    # AWAIT PATTERN 2: Using the result from previous await
    # We can use the result from the previous await as input to the next
    print("Step 2: Starting data processing...")
    result = await process_data(data)
    print(f"Step 2 complete: {result}")
    
    print(f"Final result: {result}")
    
    # RETURN IN ASYNC FUNCTION:
    # This return works like any function return
    # The caller who awaits this function will get this value
    return result

In [11]:
await basic_coroutine_example()

Step 1: Starting data fetch...
Fetching data from https://api.example.com...
Step 1 complete: Data from https://api.example.com
Step 2: Starting data processing...
Processing: Data from https://api.example.com
Step 2 complete: Processed: Data from https://api.example.com
Final result: Processed: Data from https://api.example.com


'Processed: Data from https://api.example.com'

In [12]:
# 下面只能在non-interactive environments执行
# If you want to run this in a script, you need to use asyncio.run()
asyncio.run(basic_coroutine_example())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
# DEMONSTRATION OF WHAT HAPPENS WITHOUT AWAIT:
async def demonstrate_without_await():
    """Shows what happens when you forget to use await."""
    
    print("\n--- Demonstrating the importance of await ---")
    
    # WITHOUT AWAIT - This is usually a mistake!
    # This gets a coroutine object, not the actual result
    coroutine_obj = fetch_data("https://example.com")
    print(f"Without await: {coroutine_obj}")
    print(f"Type: {type(coroutine_obj)}")
    
    # You need to clean up unused coroutines to avoid warnings
    # coroutine_obj.close()
    
    # WITH AWAIT - This gets the actual result
    actual_result = await fetch_data("https://example.com")
    print(f"With await: {actual_result}")
    print(f"Type: {type(actual_result)}")

In [21]:
await demonstrate_without_await()


--- Demonstrating the importance of await ---
Without await: <coroutine object fetch_data at 0x107037680>
Type: <class 'coroutine'>
Fetching data from https://example.com...
With await: Data from https://example.com
Type: <class 'str'>


/var/folders/d0/dmh_9hp96hbdlrk_t5jh65mw0000gp/T/ipykernel_28084/4146570258.py:1: RuntimeWarning: coroutine 'fetch_data' was never awaited
  await demonstrate_without_await()


In [23]:
# NOTE: EXPLANATION OF THE EVENT LOOP'S ROLE: ⭐️
async def explain_event_loop():
    """
    Demonstrates how the event loop enables concurrency.
    
    The event loop is the heart of async programming:
    - It runs one coroutine at a time
    - When a coroutine hits 'await', it pauses and the loop runs another
    - This creates the illusion of multiple things happening at once
    """
    
    print("\n--- Event Loop Demonstration ---")
    print("This coroutine will pause multiple times...")
    
    print("Before first await")
    await asyncio.sleep(0.1)  # Pause here - event loop can run other code
    print("After first await")
    
    print("Before second await") 
    await asyncio.sleep(0.1)  # Pause here again
    print("After second await")
    
    print("Coroutine complete!")
    
    return "Event loop demo finished"

await explain_event_loop()


--- Event Loop Demonstration ---
This coroutine will pause multiple times...
Before first await
After first await
Before second await
After second await
Coroutine complete!


'Event loop demo finished'

In [26]:
def get_loop() -> asyncio.AbstractEventLoop:
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    return loop

In [38]:
# Class with sync and async methods - MULTIPLE SOLUTIONS FOR JUPYTER

class RAG:
    """Sync method is based on async method."""

    def similarity_search(self, query: str) -> str:
        """
        Synchronous method that calls the async method.
        
        SOLUTION 1: Use asyncio.create_task() in Jupyter/IPython
        This works because Jupyter already has a running event loop.
        """
        import asyncio
        
        # Check if we're in an interactive environment (Jupyter/IPython)
        try:
            # If there's already a running loop, use create_task
            loop = asyncio.get_running_loop()
            
            # Create a task and get the result using a nested approach
            # This is a common pattern for Jupyter environments
            import nest_asyncio
            nest_asyncio.apply()  # This allows nested event loops
            
            # Now we can use run_until_complete
            return loop.run_until_complete(self.async_similarity_search(query))
            
        except RuntimeError:
            # No running loop, we can create our own
            return asyncio.run(self.async_similarity_search(query))
    
    def similarity_search_v2(self, query: str) -> str:
        """
        SOLUTION 2: Better approach using asyncio.create_task() and asyncio.gather()
        This is more Jupyter-friendly without requiring nest_asyncio
        """
        import asyncio
        
        try:
            # If there's a running loop, we need a different approach
            loop = asyncio.get_running_loop()
            
            # We can't use run_until_complete on a running loop
            # Instead, we'll need to use a different pattern
            
            # For now, let's raise a helpful error with instructions
            raise RuntimeError(
                "Cannot run sync method from async context. "
                "Use 'await rag.async_similarity_search(query)' instead, "
                "or use the async_to_sync_jupyter() helper method."
            )
            
        except RuntimeError as e:
            if "already running" in str(e) or "Cannot run" in str(e):
                raise e
            # No running loop, safe to use asyncio.run
            return asyncio.run(self.async_similarity_search(query))
    
    async def async_similarity_search(self, query: str) -> str:
        """
        Async method that performs a similarity search.
        
        This method can be awaited, allowing it to run concurrently with other tasks.
        """
        # Simulate an async operation
        await asyncio.sleep(1)
        return f"Async search result for '{query}'"
    
    def async_to_sync_jupyter(self, query: str) -> str:
        """
        SOLUTION 3: Helper method specifically for Jupyter environments
        This uses a thread-based approach to avoid event loop conflicts.
        """
        import asyncio
        import concurrent.futures
        import threading
        
        def run_in_thread():
            # Create a new event loop in a separate thread
            new_loop = asyncio.new_event_loop()
            asyncio.set_event_loop(new_loop)
            try:
                return new_loop.run_until_complete(self.async_similarity_search(query))
            finally:
                new_loop.close()
        
        # Run the async function in a separate thread
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(run_in_thread)
            return future.result()

# Let's test the working solutions
print("Testing RAG class solutions...")
rag = RAG()

print("\n=== Testing SOLUTION 3 (Thread-based approach) ===")
try:
    result = rag.async_to_sync_jupyter("example query")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n=== Testing direct async call (RECOMMENDED for Jupyter) ===")
print("Use: await rag.async_similarity_search('query') - see next cells")

Testing RAG class solutions...

=== Testing SOLUTION 3 (Thread-based approach) ===
✅ Success: Async search result for 'example query'

=== Testing direct async call (RECOMMENDED for Jupyter) ===
Use: await rag.async_similarity_search('query') - see next cells
✅ Success: Async search result for 'example query'

=== Testing direct async call (RECOMMENDED for Jupyter) ===
Use: await rag.async_similarity_search('query') - see next cells


In [39]:
# RECOMMENDED APPROACH FOR JUPYTER: Use async methods directly

print("=== BEST PRACTICE: Use async methods directly in Jupyter ===")

# This is the cleanest and most efficient approach in Jupyter/IPython
result = await rag.async_similarity_search("example query")
print(f"✅ Direct async call result: {result}")

print("\n=== You can also use asyncio.gather for multiple concurrent calls ===")
results = await asyncio.gather(
    rag.async_similarity_search("query 1"),
    rag.async_similarity_search("query 2"),
    rag.async_similarity_search("query 3")
)
print(f"✅ Concurrent results: {results}")

=== BEST PRACTICE: Use async methods directly in Jupyter ===
✅ Direct async call result: Async search result for 'example query'

=== You can also use asyncio.gather for multiple concurrent calls ===
✅ Direct async call result: Async search result for 'example query'

=== You can also use asyncio.gather for multiple concurrent calls ===
✅ Concurrent results: ["Async search result for 'query 1'", "Async search result for 'query 2'", "Async search result for 'query 3'"]
✅ Concurrent results: ["Async search result for 'query 1'", "Async search result for 'query 2'", "Async search result for 'query 3'"]


In [40]:
# SOLUTION 1: Using nest_asyncio (requires installation)
# Uncomment the following line to install nest_asyncio if needed
# !pip install nest_asyncio

try:
    import nest_asyncio
    nest_asyncio.apply()
    
    print("=== Testing SOLUTION 1 (nest_asyncio approach) ===")
    
    # Now the original similarity_search should work
    result = rag.similarity_search("example query with nest_asyncio")
    print(f"✅ nest_asyncio result: {result}")
    
except ImportError:
    print("❌ nest_asyncio not installed. Install with: pip install nest_asyncio")
    print("📝 Note: nest_asyncio allows nested event loops in Jupyter")
except Exception as e:
    print(f"❌ Error with nest_asyncio approach: {e}")

=== Testing SOLUTION 1 (nest_asyncio approach) ===
✅ nest_asyncio result: Async search result for 'example query with nest_asyncio'
✅ nest_asyncio result: Async search result for 'example query with nest_asyncio'


## Summary: Running Async Code in Jupyter/IPython

### ✅ **RECOMMENDED APPROACHES:**

1. **Direct async/await (BEST)**: Use `await rag.async_similarity_search(query)` directly in Jupyter cells
   - ✅ Clean and efficient
   - ✅ Natural async programming
   - ✅ Works out of the box

2. **nest_asyncio**: Install `nest_asyncio` and call `nest_asyncio.apply()`
   - ✅ Allows sync methods to work
   - ⚠️ Requires additional dependency
   - ⚠️ Can mask async design issues

3. **Thread-based approach**: Use `async_to_sync_jupyter()` method
   - ✅ No additional dependencies
   - ⚠️ More complex and less efficient
   - ⚠️ Loses concurrency benefits

### ❌ **WHAT DOESN'T WORK IN JUPYTER:**
- `asyncio.run()` - Creates new event loop (conflicts with Jupyter's loop)
- `loop.run_until_complete()` on running loop - Cannot run on already running loop

### 🎯 **WHY THIS HAPPENS:**
Jupyter/IPython runs its own event loop, so:
- `asyncio.get_running_loop()` returns Jupyter's loop
- `loop.run_until_complete()` fails because the loop is already running
- Direct `await` works because it uses the existing loop

### 💡 **DESIGN RECOMMENDATION:**
In modern async Python applications:
- Prefer async methods as the primary interface
- Provide sync wrappers only when necessary for compatibility
- In Jupyter, embrace the async nature and use `await` directly